In [3]:
import numpy as np

# These params come from the D1_merged_all_cracks_6micron_griddata.vtk file in /data. They are 1 less than the 
#  DIMENSIONS specified because we are using the center of each voxel.

def get_voxel_centers_as_array(num_x_pts, num_y_pts, num_z_pts, increment, file):
    total_voxels = num_x_pts * num_y_pts * num_z_pts
    natee = 0
    next_print = total_voxels / 10
    
    voxel_centers = np.empty((num_x_pts, num_y_pts, num_z_pts), dtype=int)
    with open(file, 'r') as voxel_centers_file:
        voxel_centers_file.readline()
        for line in voxel_centers_file:
            tokens = line.split(',')
            x = int(float(tokens[0]))
            y = int(float(tokens[1]))
            z = int(float(tokens[2]))
            grain_id = int(tokens[3])
            
            # We divide the voxel_centers by the increment so that we fill every index of the list
            voxel_centers[x // increment][y // increment][z // increment] = grain_id
            
            # Print progress
            natee += 1
            if natee >= next_print:
                print('Progress: ', natee / total_voxels)
                next_print += total_voxels / 10
    return voxel_centers

In [4]:
# Time: ~2 minutes
# These params come from the D1_merged_all_cracks_6micron_griddata.vtk file in /data. They are 1 less than the 
#  DIMENSIONS specified because we are using the center of each voxel.

voxel_centers = get_voxel_centers_as_array(201, 376, 301, 2, '../../data/1-voxel-centers.csv')
print('done')

Progress:  0.10000001758367279
Progress:  0.20000003516734557
Progress:  0.3000000087918364
Progress:  0.4000000263755092
Progress:  0.5
Progress:  0.6000000175836728
Progress:  0.7000000351673455
Progress:  0.8000000087918364
Progress:  0.9000000263755091
done


In [5]:
# Reads in the crack_front_points file. dadN3Dline is the output we're concerned with.

def get_crack_front_points(input_file):
    with open(input_file, 'r') as crack_points_file:
        crack_points_file.readline()
        crack_points = []
        for line in crack_points_file:
            tokens = line.split(',')
            crack_id = tokens[0]
            theta = tokens[1]
            x = tokens[5]
            y = tokens[6]
            z = tokens[7].strip()
            dadN3Dline = tokens[3]
            if int(crack_id) > 1: # Ignore the first point
                crack_points.append((crack_id, theta, x, y, z, dadN3Dline))
    return crack_points

In [6]:
# Gets the crack front points from file

crack_front_points = get_crack_front_points('../../data/0-crack_front_growth_rates_1500ppcf_transformed.csv')

In [1]:
# This will give the nearest voxel center (x,y,z) based on the voxel size being the length of 1 side of the voxel

def find_nearest_voxel_center(x, y, z, voxel_size):
    # This works because / gives a float while // gives an int. So this says that it's in one boundary until the next
    voxel_x = int(((x + (voxel_size / 2)) // voxel_size) * voxel_size)
    voxel_y = int(((y + (voxel_size / 2)) // voxel_size) * voxel_size)
    voxel_z = int(((z + (voxel_size / 2)) // voxel_size) * voxel_size)
    return voxel_x, voxel_y, voxel_z

In [8]:
# Get the index in voxel_centers from x,y,z

def get_voxel_centers_index(x, y, z, voxel_size):
    return x // voxel_size, y // voxel_size, z // voxel_size

In [25]:
# Get the x,y,z from voxel_centers indices

def get_xyz_from_voxel_centers_index(xi, yi, zi, voxel_size):
    return xi * voxel_size, yi * voxel_size, zi * voxel_size

In [34]:
# The 4 rightmost digits of the grain_id identify it. The other digits are to show which crack front it's on and 
#  whether it's above or below the crack front

def get_actual_grain_id(long_grain_id):
    return long_grain_id % 10000

In [35]:
nearest_voxels = []
voxel_size = 2

for crack_front_point in crack_front_points:
    x = float(crack_front_point[2])
    y = float(crack_front_point[3])
    z = float(crack_front_point[4])
    
    voxel_center = find_nearest_voxel_center(x, y, z, voxel_size)
    voxel_center_index = get_voxel_centers_index(voxel_center[0], voxel_center[1], voxel_center[2], voxel_size)
    grain_id = voxel_centers[int(voxel_center_index[0])][int(voxel_center_index[1])][int(voxel_center_index[2])]
    grain_id = get_actual_grain_id(grain_id)
    
    voxel_center_xyz = get_xyz_from_voxel_centers_index(voxel_center_index[0], voxel_center_index[1], voxel_center_index[2], voxel_size)
    nearest_voxels.append((voxel_center_xyz[0], voxel_center_xyz[1], voxel_center_xyz[2], grain_id))

In [36]:
with open('../../data/vis/nearest-voxel-center.csv', 'w') as w:
    w.write('X, Y, Z, Grain ID\n')
    for nearest_voxel in nearest_voxels:
        w.write('{}, {}, {}, {}\n'.format(nearest_voxel[0], nearest_voxel[1], nearest_voxel[2], nearest_voxel[3]))

In [32]:
nearest_voxels[0]

(0, 332, 306, 6310260)

In [3]:
find_nearest_voxel_center(25.607, 332.04, 309, 2)

(26, 332, 310)